# From Decision Trees to Random Forests

```
Authors: Alexandre Gramfort
         Thomas Moreau
```

## Bagging classifiers

We saw that by increasing the depth of the tree, we are going to get an over-fitted model. A way to bypass the choice of a specific depth it to combine several trees together.

Let's start by training several trees on slightly different data. The slightly different dataset could be generated by randomly sampling with replacement. In statistics, this called a boostrap sample. We will use the iris dataset to create such ensemble and ensure that we have some data for training and some left out data for testing.

In [1]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

Before to train several decision trees, we will run a single tree. However, instead to train this tree on `X_train`, we want to train it on a bootstrap sample. You can use the `np.random.choice` function sample with replacement some index. You will need to create a sample_weight vector and pass it to the `fit` method of the `DecisionTreeClassifier`. We provide the `generate_sample_weight` function which will generate the `sample_weight` array.

In [2]:
def bootstrap_idx(X):
    indices = np.random.choice(
        np.arange(X.shape[0]), size=X.shape[0], replace=True
    )
    return indices

In [3]:
bootstrap_idx(X_train)

array([ 21, 101, 100,  33, 101,   6,  78,  10,  99,  78,  99,  21,  15,
       103,  53,  95, 110,  45,  57,  28,  16,  81, 110,  17,   5,  82,
         3,   2,  15,  52,  86,  56, 107,   0, 109,  87,  30,  41,  89,
        80, 102,  19,   3, 102,  35,  73,  35, 102,  96, 101,  64,  69,
        64,  26,  32,  37,  56, 103,  68,  52, 105,  48,  73, 101,  11,
        25,  33,  24,  38,  66,  29,  19,  19,  44,  93,  68,  25,  70,
        97,  37,  46,   2,  10,  54,  67,  28, 107,  40,  56,  16,  49,
        61,  69, 102,  13,  11,  42,  43,  96,   9,   0,  67,  99,  82,
        89,  62,  69, 110,  82,  89,  43,  37])

In [4]:
from collections import Counter
Counter(bootstrap_idx(X_train))

Counter({16: 1,
         23: 3,
         57: 1,
         18: 1,
         44: 1,
         26: 2,
         55: 3,
         6: 4,
         71: 2,
         35: 1,
         66: 3,
         89: 1,
         17: 3,
         96: 1,
         34: 1,
         43: 1,
         104: 1,
         77: 1,
         67: 2,
         58: 4,
         84: 1,
         21: 2,
         52: 2,
         62: 2,
         14: 1,
         30: 2,
         54: 3,
         92: 1,
         29: 4,
         47: 2,
         99: 1,
         22: 2,
         19: 2,
         94: 2,
         70: 2,
         40: 1,
         32: 2,
         61: 3,
         5: 1,
         65: 2,
         20: 1,
         98: 3,
         28: 2,
         78: 1,
         24: 1,
         25: 2,
         3: 1,
         111: 2,
         63: 3,
         39: 1,
         1: 1,
         105: 1,
         95: 2,
         38: 1,
         87: 1,
         51: 1,
         107: 1,
         41: 3,
         101: 1,
         27: 1,
         48: 1,
         86: 1,
       

In [5]:
def bootstrap_sample(X, y):
    indices = bootstrap_idx(X)
    return X[indices], y[indices]

In [6]:
X_train_bootstrap, y_train_bootstrap = bootstrap_sample(X_train, y_train)

In [7]:
print(f'Classes distribution in the original data: {Counter(y_train)}')
print(f'Classes distribution in the bootstrap: {Counter(y_train_bootstrap)}')

Classes distribution in the original data: Counter({2: 38, 0: 37, 1: 37})
Classes distribution in the bootstrap: Counter({0: 41, 1: 37, 2: 34})


<div class="alert alert-success">
    <b>EXERCISE: Create a bagging classifier</b>:<br>
    <br>
    A bagging classifier will train several decision tree classifiers, each of them on a different bootstrap sample.
     <ul>
      <li>
      Create several <code>DecisionTreeClassifier</code> and store them in a Python list;
      </li>
      <li>
      Loop over these trees and <code>fit</code> them by generating a bootstrap sample using <code>bootstrap_sample</code> function;
      </li>
      <li>
      To predict with this ensemble of trees on new data (testing set), you can provide the same set to each tree and call the <code>predict</code> method. Aggregate all predictions in a NumPy array;
      </li>
      <li>
      Once the predictions available, you need to provide a single prediction: you can retain the class which was the most predicted which is called a majority vote;
      </li>
      <li>
      Finally, check the accuracy of your model.
      </li>
    </ul>
</div>

In [12]:
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import mode

model = [DecisionTreeClassifier() for i in range(10)]
pred_array = []
for tree in model:
    X_train_bootstrap, y_train_bootstrap = bootstrap_sample(X_train, y_train)
    tree.fit(X_train_bootstrap, y_train_bootstrap)
    pred = tree.predict(X_test)
    pred_array.append(pred)
pred_array = np.asarray(pred_array)


<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own bagging classifier, use a <code>BaggingClassifier</code> from scikit-learn to fit the above data.
</div>

In [14]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(
    estimator = DecisionTreeClassifier(),
    n_estimators = 10
)
clf.fit(X_train, y_train).score(X_test, y_test)

TypeError: __init__() got an unexpected keyword argument 'estimator'

## Random Forests

A very famous classifier is the random forest classifier. It is similar to the bagging classifier. In addition of the bootstrap, the random forest will use a subset of features (selected randomly) to find the best split.

<div class="alert alert-success">
    <b>EXERCISE: Create a random forest classifier</b>:
    <br>
    Use your previous code which was generated several <code>DecisionTreeClassifier</code>. Check the list of the option of this classifier and modify one of the parameters such that only the $\sqrt{F}$ features are used for the splitting. $F$ represents the number of features in the dataset.
</div>

In [13]:
from sklearn.tree import DecisionTreeClassifier

list_tree = [DecisionTreeClassifier() for i in range(10)]

pred_array = []
for tree in list_tree:
    X_train_bootstrap, y_train_bootstrap = bootstrap_sample(X_train, y_train)
    idx_features = np.random.choice(X.shape[1], size=int(np.sqrt(X.shape[1])), replace=False)
    X_train_bootstrap = X_train_bootstrap[:, idx_features]
    tree.fit(X_train_bootstrap, y_train_bootstrap)

    pred = tree.predict(X_test[:, idx_features])
    pred_array.append(pred)

pred_array = np.asarray(pred_array)
y_pred = mode(pred_array, axis=0).mode[0]
(y_pred == y_test).mean()

0.9736842105263158

<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own random forest classifier, use a <code>RandomForestClassifier</code> from scikit-learn to fit the above data.
</div>

In [ ]:
from figures import plot_forest_interactive
plot_forest_interactive()